In [ ]:
#function check เรียนไปแล้วหรือยัง (ตาราง subject)
from app_users.models import Subjects

def check_pass_subject(request, user_id, sub_id):
    data = Subjects.objects.filter(userid = user_id) #เอาทุกอย่างในตารางsubjectsของ Userคนนั้น
    p_sub = data.values_list('real_subject_id', flat=True) #เอาเฉพาะวิชา
    for i in p_sub:
        if sub_id in i :
            return False
    else :
        return True

In [11]:
#function check เรียนไปแล้วหรือยัง (ตาราง subject)
import sqlite3

def check_pass_subject(sub_id,u_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    #เลือกวิชาทั้งหมดที่มีจาก subjects ของ user คนนั้น
    c.execute("SELECT real_subject_id FROM subjects WHERE UserID = ?", (u_id,))
    data = c.fetchall()
    print(data)

    # Close the connections
    conn.close()
    
    for i in data:
        if sub_id in i :
            return False
    else:
        return True

check_pass_subject("80103001" , "1")

[('10113030',), ('10123102',), ('40203111',), ('40313005',), ('40313006',), ('80103001',), ('80303502',), ('80303601',), ('10113138',), ('10113139',), ('10123103',), ('40003004',), ('40203112',), ('40313007',), ('40313008',), ('80103002',), ('10123105',), ('10123106',), ('10123107',), ('10123108',), ('10123131',), ('10123133',), ('80103034',)]


False

In [ ]:
from app_users.models import Subjects

def check_credit(request, user_id, sub_id):
    data = Subjects.objects.filter(userid = user_id) #เอาทุกอย่างในตารางsubjectsของ Userคนนั้น
    p_sub = data.values_list('credit', flat=True) #เอาเฉพาะหน่วยกิต
    total_credits = 0
    for i in p_sub:
        total_credits+=i
    
    data2 = Subjects.objects.filter(real_subject_id = sub_id)#เอาSubjectที่userที่เลือก
    p_sub2 = data.values_list('credit', flat=True)#เอาหน่วยกิต
    subject_credits = p_sub2[0]
    credits_now = total_credits + subject_credits
    print(credits_now)
    if credits_now > 22 :
        return False
    else :
        return True

In [14]:
#Function check credit
import sqlite3
def check_credit(user_id,sub_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    # Execute the SELECT statement and retrieve the total sum of credits
    c.execute("SELECT SUM(credit) FROM subjects WHERE UserID = ?",(user_id,))
    result = c.fetchone()

    # Get the total sum of credits from the result tuple
    total_credits = result[0]

    # Print the total sum of credits
    print("Total credits:", total_credits)

    # Execute the SELECT statement to retrieve the sum of credits for the specified subject
    c.execute("SELECT credit FROM app_select_subjects_test_date WHERE code = ?", (sub_id,))
    result = c.fetchone()
    print(result)
    # Get the sum of credits for the specified subject from the result tuple
    subject_credits = result[0]

    # Print the sum of credits for the specified subject
    print(f"Subject {sub_id} credits:", subject_credits)

    # Close the connections
    conn.close()
    
    credits_now = total_credits + subject_credits

    if credits_now >22:
        return False
    else:
        return True

check_credit(1,"010113138")


Total credits: 57
(3,)
Subject 010113138 credits: 3


False

In [ ]:
#function check ลงไปแล้วรึยัง (ตารางเก็บวิชาที่ผู้ใช้เลือก)

In [7]:
# add 0 front of subject_code
import sqlite3
user_id = 8
conn = sqlite3.connect("w3.db")
c = conn.cursor()
c.execute("SELECT * FROM subjects WHERE UserID = ?",(user_id,))
result = c.fetchall()
for a in result:
    x = '0'+a[1]
    print(x)
    c.execute("UPDATE subjects SET real_subject_id = ? WHERE UserID = ? AND real_subject_id = ? ",(x,user_id,a[1],))

conn.commit()
conn.close()

010113030
010123102
040203111
040313005
040313006
080103001
080303502
080303601
010113138
010113139
010123103
040003004
040203112
040313007
040313008
080103002
010123105
010123106
010123107
010123108
010123131
010123133
080103034


In [ ]:
#function check วันเวลา
from datetime import datetime

In [ ]:
from datetime import datetime

event1_start = datetime(2023, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2023, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [ ]:
from datetime import datetime

event1_start = datetime(2566, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2566, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [ ]:
#function check วันเวลา
def check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    if event1_end <= event2_start or event2_end <= event1_start:
        return False
    else:
        return True

if check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    print("The two events overlap.")
else:
    print("The two events do not overlap.")
